# protein scaffolding of insulin receptor 3W11


In [1]:
# check path
import sys
print(sys.path)

['/home/yuan/anaconda3/envs/SE3nv/lib/python39.zip', '/home/yuan/anaconda3/envs/SE3nv/lib/python3.9', '/home/yuan/anaconda3/envs/SE3nv/lib/python3.9/lib-dynload', '', '/home/yuan/anaconda3/envs/SE3nv/lib/python3.9/site-packages', '/home/yuan/anaconda3/envs/SE3nv/lib/python3.9/site-packages/se3_transformer-1.0.0-py3.9.egg', '/home/yuan/data/RFdiffusion']


In [2]:
# Check available GPUs
import torch

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Found GPU with device_name {device_name}. Will run RFdiffusion on {device_name}")
else:
    print("///// NO GPU DETECTED! Falling back to CPU /////")

Found GPU with device_name NVIDIA GeForce RTX 3060. Will run RFdiffusion on NVIDIA GeForce RTX 3060


In [251]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import py3Dmol

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. prepare inputs

### 1.1 pdb

In [55]:
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open('pdb/3W11.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [54]:
from process_pdb import ProcessPdb

p = ProcessPdb('./pdb')
p.load_structure('3W11.pdb')
p.get_chains()
p.export_fasta('3W11.fasta')

Chain=A, AA residues=21
Chain=B, AA residues=15
Chain=C, AA residues=118
Chain=D, AA residues=114
Chain=E, AA residues=291
Chain=F, AA residues=11
Chain=G, AA residues=4
Chain=H, AA residues=3
Try to create ./pdb/3W11.fasta...


### 1.2 insulin receptor chain A and B

In [26]:
from process_pdb import ProcessPdb

p = ProcessPdb('./pdb')
p.load_structure('3W11.pdb')

# get chain A
chain_id = 'A'
p.split_by_chain(chain_id)

# to fasta
p.load_structure(f'3W11_{chain_id}.pdb')
p.get_chains()
p.export_fasta(f'3W11_{chain_id}.fasta')

Saved chain A with metadata to ./pdb/3W11_A.pdb
Chain=A, AA residues=21
Try to create ./pdb/3W11_A.fasta...


In [27]:
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open('pdb/3W11_A.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [28]:
from process_pdb import ProcessPdb

p = ProcessPdb('./pdb')
p.load_structure('3W11.pdb')

# get chain B
chain_id = 'B'
p.split_by_chain(chain_id)

# to fasta
p.load_structure(f'3W11_{chain_id}.pdb')
p.get_chains()
p.export_fasta(f'3W11_{chain_id}.fasta')

Saved chain B with metadata to ./pdb/3W11_B.pdb
Chain=B, AA residues=15
Try to create ./pdb/3W11_B.fasta...


In [29]:
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open('pdb/3W11_B.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [245]:
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
# Filter out the specific warning
warnings.simplefilter("ignore", PDBConstructionWarning)

from Bio.PDB import PDBParser

parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure("protein", 'pdb/insulin_target.pdb')

# Get all CA atoms from Chain A
ca_atoms = []
for model in structure:
    for chain in model:
        for residue in chain:
             # Check if residue has a CA atom
            if "CA" in residue: 
                ca_atoms.append(residue)
ca_atoms[0]

<Residue GLU het=  resseq=1 icode= >

### 1.3 guided inputs: *_ss.pt and *_adj.pt

In [250]:
# build ss.pt
from guide_input import GuideInput
from process_pdb import ProcessPdb

# retrieve residues of chain 
pp = ProcessPdb('./pdb')
pp.load_structure('3W11_B.pdb')
alpha_atoms = pp.parse_alpha_atoms('B')

# build SS
guider = GuideInput(alpha_atoms, './pdb/3W11_B')
guider.create_ss( max_mask=0)

####ss: tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2])
## transitions: [ 0  0 11 11]
####ss_argmax: tensor([2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2.])


tensor([2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2.])

In [248]:
# confirm pytorch file: secondary structure (SS)
ss = torch.load('pdb/3W11_B_ss.pt')
print(ss.shape)
ss

torch.Size([15])


tensor([2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2.])

In [249]:
# confirm pytorch file: secondary structure (SS)
ss.old = torch.load('pdb/3W11_B_ss.pt.old')
print(ss.old.shape)
ss.old

torch.Size([15])


tensor([2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2.])

In [269]:
# build adj.pt
from guide_input import GuideInput
from process_pdb import ProcessPdb

# retrieve residues of chain 
pp = ProcessPdb('./pdb')
pp.load_structure('3W11_B.pdb')
alpha_atoms = pp.parse_alpha_atoms('B')

# build SS
guider = GuideInput(alpha_atoms, './pdb/3W11_B')
guider.create_adj(exclude_loops=True)

####ss: tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2])
###segments; [(tensor(0), 1, 12)]


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0.,

In [264]:
# confirm pytorch file: secondary structure (SS)
adj = torch.load('pdb/3W11_B_adj.pt')
print(adj.shape)
adj

torch.Size([15, 15])


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0.,

In [265]:
adj.old = torch.load('pdb/3W11_B_adj.pt.old')
print(adj.old.shape)
adj.old

torch.Size([15, 15])


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0.,

### 2.3 configurations and run

In [270]:
import yaml
from pathlib import Path
from omegaconf import DictConfig, OmegaConf

# load default parameters
args = yaml.safe_load(Path('./base.yaml').read_text())
conf = OmegaConf.create(args)

chain_name = '3W11_B'
conf.inference.num_designs = 10
conf.inference.output_prefix = f'outputs/{chain_name}/scaffold'

# specify the target protein
conf.scaffoldguided.target_pdb = True
conf.scaffoldguided.target_path = 'pdb/3W11_B.pdb'
# the target with secondary structuree
conf.scaffoldguided.target_ss = f'pdb/{chain_name}_ss.pt'
# block adjacency input
conf.scaffoldguided.target_adj = f'pdb/{chain_name}_adj.pt'
# specify the fold of the protein
conf.scaffoldguided.scaffoldguided = True
# different scaffords providing scaffold_list
conf.scaffoldguided.scaffold_dir = './ppi_scaffolds'
# the binding residues 
conf.ppi.hotspot_res = ['B8', 'B12', 'B22','B23', 'B24',]

# reduce the noise added during inference to 0
conf.denoiser.noise_scale_ca = 0
conf.denoiser.noise_scale_frame = 0

In [271]:
# Initialize sampler and target/contig.
from rfdiffusion.inference import utils as iu

sampler = iu.sampler_selector(conf)

Reading models from /home/yuan/data/RFdiffusion/rfdiffusion/inference/../../models
This is inf_conf.ckpt_path
/home/yuan/data/RFdiffusion/rfdiffusion/inference/../../models/Complex_Fold_base_ckpt.pt
Assembling -model, -diffuser and -preprocess configs from checkpoint
USING MODEL CONFIG: self._conf[model][n_extra_block] = 4
USING MODEL CONFIG: self._conf[model][n_main_block] = 32
USING MODEL CONFIG: self._conf[model][n_ref_block] = 4
USING MODEL CONFIG: self._conf[model][d_msa] = 256
USING MODEL CONFIG: self._conf[model][d_msa_full] = 64
USING MODEL CONFIG: self._conf[model][d_pair] = 128
USING MODEL CONFIG: self._conf[model][d_templ] = 64
USING MODEL CONFIG: self._conf[model][n_head_msa] = 8
USING MODEL CONFIG: self._conf[model][n_head_pair] = 4
USING MODEL CONFIG: self._conf[model][n_head_templ] = 4
USING MODEL CONFIG: self._conf[model][d_hidden] = 32
USING MODEL CONFIG: self._conf[model][d_hidden_templ] = 32
USING MODEL CONFIG: self._conf[model][p_drop] = 0.15
USING MODEL CONFIG: sel

In [272]:
# unconditional
type(sampler)

rfdiffusion.inference.model_runners.ScaffoldedSampler

In [273]:
# Loop over number of designs to sample.
import glob

# default is zero
design_startnum = sampler.inf_conf.design_startnum
if sampler.inf_conf.design_startnum == -1:
    existing = glob.glob(sampler.inf_conf.output_prefix + "*.pdb")
    indices = [-1]
    for e in existing:
        print(e)
        m = re.match(".*_(\d+)\.pdb$", e)
        print(m)
        if not m:
            continue
        m = m.groups()[0]
        indices.append(int(m))
    design_startnum = max(indices) + 1
conf.inference.design_startnum = design_startnum

In [274]:
# run model
from run_rfd import RunRfd

c = RunRfd(sampler)
c.run(overwrite=True)

Making design outputs/3W11_B/scaffold_0
Scaffold constrained based on file:  HHH_b2_02136
With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696048699785024
GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCGSHLVEALYLVCGE
meta data: outputs/3W11_B/scaffold_0.trb
Finished design in 0.28 minutes
Making design outputs/3W11_B/scaffold_1
Scaffold constrained based on file:  b3b12a33d5fcf2d91a224f40ea3c82bf_0001
With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696048699785024
GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCGSHLVEALYLVCGE
meta data: outputs/3W11_B/scaffold_1.trb
Finished design in 0.33 minutes
Making design outputs/3W11_B/scaffold_2
Scaffold constrained based on file:  HHH_b1_03900
With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696048699785024
GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCGSHLVEALYLVCGE
meta d

### 2.4 display results

In [275]:
# view structure of one design
c.display_pdb(9)

outputs/3W11_B/scaffold_9.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

'outputs/3W11_B/scaffold_9.pdb'

In [276]:
trb = c.from_trb(9)
trb['plddt']

array([[0.19298653, 0.22239392, 0.22722727, ..., 0.7072913 , 0.7258782 ,
        0.784853  ],
       [0.33515483, 0.33768463, 0.3331424 , ..., 0.74803305, 0.7637452 ,
        0.8357172 ],
       [0.4208433 , 0.3830545 , 0.3840518 , ..., 0.7242211 , 0.74122214,
        0.8302758 ],
       ...,
       [0.96746194, 0.9779339 , 0.9820982 , ..., 0.9889224 , 0.9861925 ,
        0.9772316 ],
       [0.97355664, 0.982401  , 0.98609936, ..., 0.9917383 , 0.9889206 ,
        0.980064  ],
       [0.97910905, 0.9861988 , 0.9895191 , ..., 0.99357986, 0.99081   ,
        0.98217183]], dtype=float32)

In [277]:
from process_pdb import ProcessPdb

p = ProcessPdb('./outputs/3W11_B')
p.load_structure('scaffold_9.pdb')
p.get_chains()

Chain=A, AA residues=55
Chain=B, AA residues=15


### 2.5 analysis